In [ ]:
#The First International Workshop on Arabic Big Data & AI (IWABigDAI) May 11 and May 12 2022
#https://sites.google.com/view/arabicbigdata/home

#Tutorial 1: Download Tweets using Tweepy (Twitter API)
#author: Dr Mahmoud El-Haj (with help from the Internet)
#GitHub repository: https://github.com/drelhaj/NLP_ML_Visualization_Tutorial

In [ ]:
from datetime import date, timedelta
import tweepy
from random import randint
from time import sleep

client = tweepy.Client(bearer_token='AAAAAAAAAAAAAAAAAAAAAOmWVwEAAAAAJlJlIkE%2BiAJ3w0AJjGwgaM4ff1g%3Drvd7wmpYm0vnGC8AP3CCNWQw8UTUcPrHgNlPY7PGS2WkppmXrQ')#you need Twitter Academic Account https://developer.twitter.com/en/products/twitter-api/academic-research



def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

import datetime        
query = "Lancaster University"
country = "place_country:GB"# add country code. QA: Qatar, GB: UK & NI ...etc
queryCombined = query + " " + country
print(queryCombined)
start_date = date(2022, 1, 28)
end_date = date(2023, 3, 13)
printHeaders = 1

for single_date in daterange(start_date, end_date):
#try:
    start_date = single_date.strftime("%Y-%m-%dT00:00:00Z")
    dtObj = datetime.datetime.strptime(start_date, "%Y-%m-%dT00:00:00Z")
    end_date = dtObj + timedelta(days=1)
    start_date_str = str(start_date).replace(" 00", "T00").replace("00 ", "00Z")
    end_date_str = str(end_date).replace(" 00", "T00")+"Z"
    print(start_date_str)
    print(end_date_str)
    downloadTweets(start_date_str,end_date_str,queryCombined,"csvs/2023-lancs.csv",client,printHeaders)
    printHeaders = 0
#except:
#    pass

    #sleep for random seconds
    sleep(randint(2,5))




In [ ]:
def downloadTweets(start_time, end_time, query, csv_file,client,printHeaders):
    import tweepy

    import tweepy
    import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
    import csv
    from urlextract import URLExtract

    extractor = URLExtract()

    with open(csv_file, 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        if printHeaders>0:
            writer.writerow(["tweet_id","tweet_text","dateTime","author_id","retweet_count","like_count","quote_count","reply_count","UserLocation","Tweet_Place","URLs", "mentions", "hashtags", "sentiment"])
            file.flush()

        #I'm getting the geo location of the tweet as well as the location of the user and setting the number of tweets returned to 10 (minimum) - Max is 100
        tweets = client.search_all_tweets(query=query, tweet_fields=['context_annotations','created_at', 'public_metrics'],
                                          place_fields=['place_type', 'geo'], user_fields=['location'], expansions='author_id,geo.place_id',
                                          start_time=start_time,
                                          end_time=end_time, max_results=100)
        
        if tweets.data is None:
            print("")
        else:
            print(len(tweets.data))

            #.flatten(limit=1000)
            # Get list of places and users
            places = {p["id"]: p for p in tweets.includes['places']}
            users = {u["id"]: u for u in tweets.includes['users']}

            #loop through the tweets to get the tweet ID, Text, Author ID, User Location, Tweet Location + public_metrics
            for tweet in tweets.data:
#                 print(tweet.id)
#                 print(tweet.text)
#                 print(tweet.created_at)
#                 print(tweet.author_id)
#                 print(tweet.public_metrics['retweet_count'])
#                 print(tweet.public_metrics['like_count'])
#                 print(tweet.public_metrics['quote_count'])
#                 print(tweet.public_metrics['reply_count'])
                userLocation = ""
                placeFullName = ""
                if users[tweet.author_id]:
                    user = users[tweet.author_id]
                    userLocation = user.location
                if places[tweet.geo['place_id']]:
                    place = places[tweet.geo['place_id']]
                    placeFullName = place.full_name
    #                 print("================")
                cleaned_tweet =clean_tweet(tweet.text)
                cleaned_tweet =cleaned_tweet.replace("\n","").replace("\r","")
                mentions = re.findall("@([a-zA-Z0-9_]{1,50})", tweet.text)
                hashtags = re.findall("#([a-zA-Z0-9_]{1,50})", tweet.text)
                urls = extractor.find_urls(tweet.text)
                writer.writerow([tweet.id,cleaned_tweet,tweet.created_at,tweet.author_id,tweet.public_metrics['retweet_count'],tweet.public_metrics['like_count'],tweet.public_metrics['quote_count'],tweet.public_metrics['reply_count'],userLocation,placeFullName,urls,mentions,hashtags,get_tweet_sentiment(cleaned_tweet)])
                file.flush()

    file.close()          


    # # Replace the limit=1000 with the maximum number of Tweets you want
    # for tweet in tweepy.Paginator(client.search_recent_tweets, query=query,
    #                               tweet_fields=['context_annotations', 'created_at'], max_results=100).flatten(limit=1000):
    #     print(tweet.id)

In [ ]:
import re # importing regex
import string
 
def clean_tweet(tweet):
    '''
    Remove unncessary things from the tweet 
    like mentions, hashtags, URL links, punctuations
    '''
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)

    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)

    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)

    # remove mentions
    tweet = re.sub(r'@[A-Za-z0-9]+', '', tweet)  

    # remove punctuations like quote, exclamation sign, etc.
    # we replace them with a space
    tweet = re.sub(r'['+string.punctuation+']+', ' ', tweet)

    return tweet 

In [ ]:
from textblob import TextBlob

def get_tweet_sentiment(tweet):

  # create TextBlob object of the passed tweet text
    blob = TextBlob(clean_tweet(tweet))
 
  # get sentiment
    if blob.sentiment.polarity > 0:
        sentiment = 'positive'
    elif blob.sentiment.polarity < 0:
        sentiment = 'negative'
    else:
        sentiment = 'neutral'
 
    return sentiment
# testing tweet sentiment
sample_tweet = "wwWhy don’t US is doing completely lockdown, @obama we are sitting home @elhaj doing #sacrifice for #ourself & community . while many of others enjoying freedom & doing shopping’s & making other people’s infected#Coronavirus"
cleaned_tweet =clean_tweet(sample_tweet)
mentions = re.findall("@([a-zA-Z0-9_]{1,50})", sample_tweet)
hashtags = re.findall("#([a-zA-Z0-9_]{1,50})", sample_tweet)
print(cleaned_tweet)
print(get_tweet_sentiment(sample_tweet)) # Output: negative
print(mentions)
print(hashtags)

In [ ]:
from urlextract import URLExtract

extractor = URLExtract()
urls = extractor.find_urls("This is a link http://www.google.com hey man how it s fgoid www.yahoo.com, lasdkfj asdf asldkfj htts://www.ommak.com lasdkfljasdkf http://hello.org")
print(urls) # prints: ['stackoverflow.com']

In [ ]:
myString = "This is a link http://www.google.com hey man how it s fgoid www.yahoo.com, lasdkfj asdf asldkfj htts://www.ommak.com lasdkfljasdkf http://hello.org"
print(re.findall("(?P<url>https?://[^\s]+)", myString))